In [19]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf

import pandas as pd
import numpy as np
from src import mlp_utility
import gitable_src.model as da_model

%aimport -tensorflow
%aimport -sklearn
%aimport -pandas, -numpy, -seaborn, -matplotlib

In [20]:
# path = '/mnt/rna_augment/'
path = '/projectbig/jupyternotebook/rnaseq_augmentation/1.0/'

df_gtex_train = pd.read_csv(path+'data/processed/tissue_fig2b/df_gtex_train_gini.csv', index_col="Unnamed: 0")
df_gtex_test = pd.read_csv(path+'data/processed/tissue_fig2b/df_gtex_test_gini.csv', index_col="Unnamed: 0")

df_gtex_train = df_gtex_train.loc[~df_gtex_train.tissue.isin(['BLADDER', 'FALLOPIAN TUBE', 'CERVIX UTERI'])].copy()
df_gtex_test = df_gtex_test.loc[~df_gtex_test.tissue.isin(['BLADDER', 'FALLOPIAN TUBE', 'CERVIX UTERI'])].copy()

df_gtex = pd.concat([df_gtex_train.copy(), df_gtex_test.copy()])
df_gtex_train_sra = df_gtex.loc[~df_gtex.tissue.isin(['BLOOD VESSEL'])].copy()

df_tcga_opt = pd.read_csv(path+'data/processed/tissue_fig2b/df_tcga_val_gini.csv', index_col="bigwig_file")
df_tcga_test = pd.read_csv(path+'data/processed/tissue_fig2b/df_tcga_test_gini.csv', index_col="bigwig_file")

df_tcga = pd.concat([df_tcga_opt.copy(), df_tcga_test.copy()])

df_sra_test = pd.read_csv(path+'data/processed/tissue_fig2b/df_sra_test_study_split_gini.csv', index_col="run_accession")
df_sra_opt = pd.read_csv(path+'data/processed/tissue_fig2b/df_sra_opt_study_split_gini.csv', index_col="run_accession")

df_sra_all = pd.concat([df_sra_test.copy(), df_sra_opt.copy()])
df_sra_all = df_sra_all.loc[~df_sra_all.tissue.isin(['BLOOD VESSEL','BLADDER', 'FALLOPIAN TUBE', 'CERVIX UTERI'])]

df_gtex_sra = pd.concat([df_gtex.copy(), df_sra_all.copy()])

In [21]:
df_train, df_test, df_sra_opt_norm, tissue_mapping = mlp_utility.reduce_classes_tiplet(df_gtex, df_tcga, df_sra_all)

In [12]:
source = pd.read_csv('example_gtex_train_data.csv', index_col='sample_id').values.astype('float32')
target = pd.read_csv('example_tcga_test_data.csv', index_col='sample_id').values.astype('float32')
bias = pd.read_csv('example_sra_train_data.csv', index_col='sample_id').values.astype('float32')

In [23]:
df_train.index.names = ['sample_id']
df_test.index.names = ['sample_id']
df_sra_opt_norm.index.names = ['sample_id']

In [25]:
df_test.sample(frac=0.3).to_csv('example_tcga_test_data.csv')

In [53]:
source_1 = pd.read_csv('example_gtex_train_data_1.csv', index_col='sample_id')
source_2 = pd.read_csv('example_gtex_train_data_2.csv', index_col='sample_id')

In [26]:
target = pd.read_csv('example_tcga_test_data.csv', index_col='sample_id')
bias = pd.read_csv('example_sra_train_data.csv', index_col='sample_id')

In [54]:
compression_opts = dict(method='zip',
                        archive_name='example_gtex_train_data_1.csv')  
source_1.to_csv('example_gtex_train_data_1.zip', compression=compression_opts)  

compression_opts = dict(method='zip',
                        archive_name='example_gtex_train_data_2.csv')  
source_2.to_csv('example_gtex_train_data_2.zip', compression=compression_opts)  

In [48]:
compression_opts = dict(method='zip',
                        archive_name='example_tcga_test_data.csv')  
bias.to_csv('example_tcga_test_data.zip', compression=compression_opts)  

In [49]:
compression_opts = dict(method='zip',
                        archive_name='example_sra_train_data.csv')  
bias.to_csv('example_sra_train_data.zip', compression=compression_opts)  

In [52]:
nrow = source.shape[0] // 2
for i in range(1,3):
    source.iloc[(i-1)*nrow:i*nrow].to_csv('example_gtex_train_data_'+str(i)+'.csv')    

In [13]:
tissue_mapping = {'ADRENAL GLAND': 0,
 'BONE MARROW': 1,
 'BRAIN': 2,
 'BREAST': 3,
 'ESOPHAGUS': 4,
 'KIDNEY': 5,
 'LIVER': 6,
 'LUNG': 7,
 'OVARY': 8,
 'PANCREAS': 9,
 'PROSTATE': 10,
 'SKIN': 11,
 'STOMACH': 12,
 'TESTIS': 13,
 'THYROID': 14,
 'UTERUS': 15}

In [14]:
config = {'source_layers': [128],
      'classifier_layers': [],
      'lr': 0.0002,
      'classes': 16,
      'batch_size': 64,
      'margin': 11,
         'print': True}

In [57]:
source.head()

,ENSG00000000005.5,ENSG00000001626.14,ENSG00000002587.9,ENSG00000002726.20,ENSG00000002745.12,ENSG00000002746.14,ENSG00000004468.12,ENSG00000004809.13,ENSG00000004846.16,ENSG00000004848.7,...,ENSG00000283567.1,ENSG00000283594.1,ENSG00000283599.1,ENSG00000283619.1,ENSG00000283632.1,ENSG00000283644.1,ENSG00000283652.1,ENSG00000283663.1,ENSG00000283683.1,tissue
sample_id,,,,,,,,,,,,,,,,,,,,,
GTEX-PWOO-1726-SM-48TDC,0.044760,0.020434,1.821738,0.043114,0.144771,0.001011,0.775638,0.080733,0.008970,3.492316,...,0.114023,0.000000,0.089422,0.000000,1.291368,0.316412,0.253174,0.271577,0.021850,8
GTEX-11OF3-0011-R8b-SM-57WCS,0.034606,0.020060,0.409162,0.003801,0.035735,0.075476,0.458522,0.005691,0.000253,0.986033,...,1.052411,0.050721,0.020332,0.015006,0.027323,0.000000,0.031109,0.023375,0.003602,2
GTEX-146FQ-1026-SM-5LU8I,0.094362,0.085092,1.455137,0.277870,0.143593,0.005118,0.244554,0.006030,0.008716,0.015720,...,0.000000,0.000000,0.048768,0.000000,0.608482,0.123684,0.129230,1.236042,0.102766,4
GTEX-13CF3-1026-SM-5LZWY,0.008815,1.948978,0.160949,0.025389,0.008619,0.001702,0.091486,0.005231,0.000830,0.312152,...,0.012537,0.000000,0.015854,0.001447,0.315391,0.000000,0.029273,0.000772,0.007857,9
GTEX-XMD1-0011-R5A-SM-4AT47,0.000000,0.048182,0.252434,0.013567,0.783192,0.044670,0.418057,0.022527,0.000000,1.879564,...,0.085011,0.000000,0.000000,0.010374,0.041341,0.000000,0.177014,0.372148,0.004887,2


In [55]:
model = da_model.DaModel(source, target, bias, tissue_mapping, config=config)

TypeError: '(slice(None, None, None), -1)' is an invalid key

In [16]:
model.train_source_mapper(epochs=10)

In [17]:
model.eval_source_mapper()

0.6127214170692431


In [18]:
model.train_da(epochs=15)

0.35507246376811596
0.5603864734299517
0.6272141706924316
0.6586151368760065
0.6690821256038647
0.7198067632850241
0.7391304347826086
0.7560386473429952
0.7737520128824477
0.7657004830917874
0.7818035426731079
0.7866344605475041
0.7962962962962963
0.8035426731078905
0.8140096618357487
